In [2]:
import merkletools
from web3 import Web3
from solcx import compile_files
import client.utils as utils
import client.ram as ram
import copy

In [3]:
w3 = Web3(Web3.HTTPProvider("http://localhost:8545"))
compiled_sol = compile_files(["matchcheck.sol"], output_values=['abi', 'bin'])['matchcheck.sol:MatchCheck']
MatchCheck = w3.eth.contract(address='0x392046542000F922E12645f24606A376B99AcE45', abi=compiled_sol['abi'])

In [53]:
code = ram.RamCoder(10)

#fibonacci
code.load(1, 0)
code.load(1, 1)
code.load(7500, 3)
code.load(1, 4)
code.load(2, 5)
code.add(0, 1, 2)
code.copy_id(4, 0)
code.copy_id(5, 1)
code.sub(3, 4, 3)
code.jnz(3, 4)
code.end()

ram1 = ram.Ram(code.source)
ram1()
ram1.rc_list

mt1 = utils.reduced_config_mt(ram1.rc_list)

In [54]:
contractors = [w3.eth.accounts[3], w3.eth.accounts[5]]

tx_hash = MatchCheck.functions.request_mh_proof(
    len(mt1.leaves), 
    '0x' + mt1.get_merkle_root(),
    contractors[0],
    contractors[1],
    '0x' + ram1.code_mt.get_merkle_root()).transact({'from': w3.eth.accounts[0]})
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

key = '0x' + utils.calculate_key(
    len(mt1.leaves), 
    '0x' + mt1.get_merkle_root(),
    contractors[0],
    contractors[1],
    '0x' + ram1.code_mt.get_merkle_root())

tx_receipt.gasUsed

186174

In [55]:
import asyncio
mt = [mt1, mt1]

c1_tx = []
c2_tx = []

def handle_event(event, conractor, tx_list, rc_list):
    n = event['args']['n']

    rc = rc_list[n]
    params = {
        "PC" : rc['PC'], 
        "OpCode" : int(rc['OpCode']), 
        "args" : rc['args'], 
        "args_values" : utils.sterilize_ints(rc['args_values']), 
        "args_proof" : rc['args_proofs'],
        "store_mt_root" : rc['store_mt_root'],
        "nextOpCode" : int(rc_list[n+1]['OpCode']),
        "nextArgs" : utils.sterilize_ints(rc_list[n+1]['args']),
        "next_op_proof" : rc['next_op_proof'],
        "key" : key}

    tx = MatchCheck.functions.send_proof(
        utils.translate_proof(mt[conractor], n),
        utils.translate_proof(mt[conractor], n+1),
        params,
        key).transact({'from': contractors[conractor]})
    tx_list.append(tx)
    print(f'conractor = {conractor} n = {n}')
    

async def log_loop_c1(poll_interval):
    old = []
    for x in range(5):
        dif = list(set(MatchCheck.events.request_proof.getLogs(fromBlock=1)) - set(old))
        old = list(set(MatchCheck.events.request_proof.getLogs(fromBlock=1)))
        for new_event in dif:
            if (key == "0x" + new_event['args']['key'].hex()
                and contractors[0] == new_event['args']['c']):
                handle_event(new_event, 0, c1_tx, ram1.rc_list)
        await asyncio.sleep(poll_interval)

async def log_loop_c2(poll_interval):
    old = []
    for x in range(5):
        dif = list(set(MatchCheck.events.request_proof.getLogs(fromBlock=1)) - set(old))
        old = list(set(MatchCheck.events.request_proof.getLogs(fromBlock=1)))
        for new_event in dif:
            if (key == "0x" + new_event['args']['key'].hex()
                and contractors[1] == new_event['args']['c']):
                handle_event(new_event, 1, c2_tx, ram1.rc_list)
        await asyncio.sleep(poll_interval)

async def main():

    await asyncio.gather(
        log_loop_c1(1),
        #log_loop_c2(1)
        )


await main()

conractor = 0 n = 15001


In [9]:
key

'0xb191941340d65220d835dcba06d9f307a6fe617d3744eb01895e27b39bd5e6a5'

In [56]:
total_cost_c1 = 0
for tx in c1_tx:
    receipt = w3.eth.waitForTransactionReceipt(tx)
    total_cost_c1 += receipt.gasUsed

total_cost_c2 = 0
for tx in c2_tx:
    receipt = w3.eth.waitForTransactionReceipt(tx)
    total_cost_c2 += receipt.gasUsed

print(total_cost_c1)
print(total_cost_c2)

208967
0


NameError: name 'c1_tx' is not defined